In [1]:
!pip install pandas numpy matplotlib seaborn pyspark scikit-learn geopandas

In [2]:
""" LIBRAIRIES IMPORTÉES """
import re

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, StringType, IntegerType


In [3]:
""" INITIALISATION """
spark = SparkSession.builder \
    .appName("CorrectionPointsRouges") \
    .master("local[*]") \
    .config("spark.driver.host", "localhost") \
    .getOrCreate()

In [4]:
""" JOINTURE DES POINTS TUNBARJO DU DATASET POINTS ROUGES """
# Chargement du dataset principal (Points Rouges)
df_pts_rouges = spark.read.option("header", "true") \
    .option("delimiter", ",") \
    .option("multiLine", "true") \
    .option("quote", "\"") \
    .option("escape", "\"") \
    .option("inferSchema", "true") \
    .csv('points_rouges_lyon_complet.csv')

# Chargement du dataset à soustraire (TUNBARJO)
df_tunbarjo = spark.read.option("header", "true") \
    .option("delimiter", ";") \
    .option("inferSchema", "true") \
    .csv("resultats_croisement_complet/TUNBARJO.csv")

# Réalisation du "Left Anti Join" (Soustraction)
# On ne garde que les lignes de pts_rouges qui n'ont PAS de correspondance (long, lat) dans tunbarjo
df_pts_rouges_raw = df_pts_rouges.join(
    df_tunbarjo, 
    on=['longitude', 'latitude'], 
    how='left_anti'
)

# Affichage des résultats 
print(f"Nombre de lignes initiales : {df_pts_rouges.count()}")
print(f"Nombre de lignes finales : {df_pts_rouges_raw.count()}")

df_pts_rouges_raw.show(15)

Nombre de lignes initiales : 24256
Nombre de lignes finales : 22498
+-----------+------------+-------------+--------------------+
|  longitude|    latitude|commune_insee|         description|
+-----------+------------+-------------+--------------------+
|4.836445341|45.758437683|        69123|                NULL|
| 4.85954302|45.731104987|        69123|                NULL|
|4.833717002|45.731325458|        69123|                NULL|
|4.867706711|45.729358056|        69123|                NULL|
|4.797329156|45.803996633|        69194|         Éboulement |
|4.863795432|45.763639697|        69123|Transformation de...|
|4.865337401|45.766321529|        69123|discontinuité des...|
| 4.87557315|45.769984946|        69266|marquages au sol ...|
| 4.87531999|45.771937705|        69266|les voitures pren...|
|4.853396287|45.741487088|        69123|Toute l'Avenue BE...|
|4.874168478|45.738423192|        69123|Rue Maryse Bastié...|
|4.869377621|45.736407856|        69123|Carrefour du Bach...|
| 

In [5]:
""" DIMENSIONS DU DATAFRAME ET VALEURS MANQUANTES PAR COLONNE """
nb_lignes = df_pts_rouges_raw.count()
nb_cols = len(df_pts_rouges_raw.columns)
print(f"Dimensions : ({nb_lignes}, {nb_cols})")

print("--- Valeurs manquantes dans df_pts_rouges après jointure avec les travaux ---")
# On convertit tout en String temporairement pour éviter les erreurs de conversion (Cast)
df_pts_rouges_raw.select([
    F.count(
        F.when(
            F.col(c).isNull() | 
            (F.col(c).cast("string") == "") | 
            (F.col(c).cast("string") == "nan") | 
            (F.col(c).cast("string") == "NaN"), 
            c
        )
    ).alias(c)
    for c in df_pts_rouges_raw.columns
]).show()

Dimensions : (22498, 4)
--- Valeurs manquantes dans df_pts_rouges après jointure avec les travaux ---
+---------+--------+-------------+-----------+
|longitude|latitude|commune_insee|description|
+---------+--------+-------------+-----------+
|        0|       0|            0|       7645|
+---------+--------+-------------+-----------+



In [6]:
""" GESTION DES VALEURS MANQUANTES """

# Supprime toute la ligne si AU MOINS UNE valeur est manquante (NULL ou NaN) dans n'importe quelle colonne
df_pts_rouges_clean = df_pts_rouges_raw.na.drop(how="any")

print(f"Nombre de lignes restantes après nettoyage : {df_pts_rouges_clean.count()}")

Nombre de lignes restantes après nettoyage : 14853


In [7]:
""" DIMENSIONS DU DATAFRAME ET VALEURS MANQUANTES PAR COLONNE """
nb_lignes = df_pts_rouges_clean.count()
nb_cols = len(df_pts_rouges_clean.columns)
print(f"Dimensions : ({nb_lignes}, {nb_cols})")

print("--- Valeurs manquantes dans df_pts_rouges après nettoyage ---")
# On convertit tout en String temporairement pour éviter les erreurs de conversion (Cast)
df_pts_rouges_clean.select([
    F.count(
        F.when(
            F.col(c).isNull() | 
            (F.col(c).cast("string") == "") | 
            (F.col(c).cast("string") == "nan") | 
            (F.col(c).cast("string") == "NaN"), 
            c
        )
    ).alias(c)
    for c in df_pts_rouges_clean.columns
]).show()

# Pour visualiser si le prétraitement a fonctionné 
df_pts_rouges_clean.show(15)

Dimensions : (14853, 4)
--- Valeurs manquantes dans df_pts_rouges après nettoyage ---
+---------+--------+-------------+-----------+
|longitude|latitude|commune_insee|description|
+---------+--------+-------------+-----------+
|        0|       0|            0|          0|
+---------+--------+-------------+-----------+

+-----------+------------+-------------+--------------------+
|  longitude|    latitude|commune_insee|         description|
+-----------+------------+-------------+--------------------+
|4.797329156|45.803996633|        69194|         Éboulement |
|4.863795432|45.763639697|        69123|Transformation de...|
|4.865337401|45.766321529|        69123|discontinuité des...|
| 4.87557315|45.769984946|        69266|marquages au sol ...|
| 4.87531999|45.771937705|        69266|les voitures pren...|
|4.853396287|45.741487088|        69123|Toute l'Avenue BE...|
|4.874168478|45.738423192|        69123|Rue Maryse Bastié...|
|4.869377621|45.736407856|        69123|Carrefour du Bach.

In [8]:
""" CONFIGURATION RÉFÉRENTIELLE """
# Les catégories de problèmes qui serviront de filtre 
CONFIG_PROBLEMES = {
    'Infrastructure manquante': {'mots': r'piste|manque|absence|discontinuité|coupure|cyclable|bande|inexistant', 'urgence': 3, 'facilite': 3},
    'Carrefours dangereux': {'mots': r'carrefour|intersection|rond-point|traversée|giratoire|priorité', 'urgence': 3, 'facilite': 3},
    'Danger & Vitesse': {'mots': r'vitesse|rapide|trop vite|ralentir|Éboulement|dangereu|sécurité|accident|(?<!angle\s)(?<!feuilles\s)mort', 'urgence': 3, 'facilite': 2},
    'Conflit Voiture/Stationnement': {'mots': r'stationnement|garé|voiture|parking|double file|sas|véhicule|portière', 'urgence': 2, 'facilite': 2},
    'Signalisation & Marquage': {'mots': r'panneau|feu|signalisation|marquage|peinture|sol|invisible|effacé', 'urgence': 1, 'facilite': 1},
    'Conflit Piétons': {'mots': r'piéton|trottoir|partagé|quai|mixité|promeneur', 'urgence': 1, 'facilite': 2},
    'Conflit Bus/TCL': {'mots': r'bus|tcl|arrêt|voie bus|angle mort', 'urgence': 2, 'facilite': 2},
    'Aménagement inadapté': {'mots': r'étroit|largeur|bordure|trottoir|poteau|obstacle', 'urgence': 2, 'facilite': 1}
}

def get_label_facilite_txt(score):
    if score <= 1: return "🟢 Facile (Quick Win)"
    if score <= 2: return "🟡 Moyen (Intervention)"
    return "🔴 Difficile (Structurel)"

conf_broadcast = spark.sparkContext.broadcast(CONFIG_PROBLEMES)

In [9]:
""" USER DEFINED FUNCTION (UDF) DE DÉTECTION ET CRÉATION DES LISTES (SANS EXPLODE) """


# Définition des UDFs pour récupérer les scores sous forme de LISTE
def detecter_themes_py(texte):
    if texte is None or texte == "": return []
    res = []
    for nom, cfg in conf_broadcast.value.items():
        if re.search(cfg['mots'], str(texte), re.IGNORECASE):
            res.append(nom)
    return res

def get_scores_urgence_py(categories):
    if not categories: return []
    scores = []
    # On utilise ton dictionnaire de config déjà existant
    config_dict = conf_broadcast.value 
    for cat in categories:
        if cat in config_dict:
            scores.append(config_dict[cat]['urgence'])
        else:
            scores.append(0)
    return scores

def get_scores_facilite_py(categories):
    if not categories: return []
    scores = []
    config_dict = conf_broadcast.value
    for cat in categories:
        if cat in config_dict:
            scores.append(config_dict[cat]['facilite'])
        else:
            scores.append(0)
    return scores

# Enregistrement des UDFs Spark
detecter_themes_udf = F.udf(detecter_themes_py, ArrayType(StringType()))
udf_get_urgence = F.udf(get_scores_urgence_py, ArrayType(IntegerType()))
udf_get_facilite = F.udf(get_scores_facilite_py, ArrayType(IntegerType()))

In [10]:
""" TRAITEMENT ET DÉTECTION DES THÈMES """
# Application : On ajoute les colonnes de listes
df_processed = df_pts_rouges_clean.withColumn(
    "problemes_list", detecter_themes_udf(F.col("description"))
).withColumn(
    "liste_urgence", 
    udf_get_urgence(F.col("problemes_list"))
).withColumn(
    "liste_facilite", 
    udf_get_facilite(F.col("problemes_list"))
)

In [11]:
""" ENRICHISSEMENT FINAL AVEC SCORES ET CATÉGORIES """
# Sélection finale des colonnes 
df_gold = df_processed.select(
    "longitude", 
    "latitude", 
    "commune_insee", 
    "description",
    F.col("problemes_list").alias("categories_detectees"), # Renommage pour clarté
    "liste_urgence",
    "liste_facilite"
)

In [12]:
""" VERIFICATION DE LA DIMENSION DU DATAFRAME ET AFFICHAGE """
nb_lignes = df_gold.count()
nb_cols = len(df_gold.columns)
print(f"Dimensions : ({nb_lignes}, {nb_cols})")
df_gold.show(10, truncate=True)

Dimensions : (14853, 7)
+-----------+------------+-------------+--------------------+--------------------+-------------+--------------+
|  longitude|    latitude|commune_insee|         description|categories_detectees|liste_urgence|liste_facilite|
+-----------+------------+-------------+--------------------+--------------------+-------------+--------------+
|4.797329156|45.803996633|        69194|         Éboulement |  [Danger & Vitesse]|          [3]|           [2]|
|4.863795432|45.763639697|        69123|Transformation de...|[Infrastructure m...| [3, 3, 2, 2]|  [3, 3, 2, 2]|
|4.865337401|45.766321529|        69123|discontinuité des...|[Infrastructure m...|          [3]|           [3]|
| 4.87557315|45.769984946|        69266|marquages au sol ...|[Infrastructure m...|    [3, 2, 1]|     [3, 2, 1]|
| 4.87531999|45.771937705|        69266|les voitures pren...|[Conflit Voiture/...|          [2]|           [2]|
|4.853396287|45.741487088|        69123|Toute l'Avenue BE...|                  [

In [13]:
""" EXPORT CSV AVEC LISTES """
print("Génération du CSV final compact...")

# Conversion en Pandas
result_pandas = df_gold.toPandas()

# Export
output_filename = "resultats_croisement_complet/points_rouges_sans_travaux_traite.csv"
result_pandas.to_csv(output_filename, index=False, encoding='utf-8-sig')

print("="*80)
print(f"TERMINÉ ! Le fichier '{output_filename}' a été créé.")
print("Aperçu des premières lignes :")
print(result_pandas[['categories_detectees', 'liste_urgence']].head(10).to_string(index=False))

Génération du CSV final compact...
TERMINÉ ! Le fichier 'resultats_croisement_complet/points_rouges_sans_travaux_traite.csv' a été créé.
Aperçu des premières lignes :
                                                                            categories_detectees liste_urgence
                                                                              [Danger & Vitesse]           [3]
[Infrastructure manquante, Carrefours dangereux, Conflit Voiture/Stationnement, Conflit Bus/TCL]  [3, 3, 2, 2]
                                                                      [Infrastructure manquante]           [3]
             [Infrastructure manquante, Conflit Voiture/Stationnement, Signalisation & Marquage]     [3, 2, 1]
                                                                 [Conflit Voiture/Stationnement]           [2]
                                                                                              []            []
                                                        